<a href="https://colab.research.google.com/github/Rn86222/colaba/blob/main/My_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#Twitter関連

!pip install python-twitter
import random
import tweepy
import csv
import twitter
import time
import sys

     |████████████████████████████████| 67 kB 4.7 MB/s 


In [2]:
#言語処理関連

!pip install dezero
!pip install janome
import numpy as np
import dezero
from dezero.datasets import Dataset
import re
from janome.tokenizer import Tokenizer
import numpy as np
import dezero
from dezero import Model
from dezero import SeqDataLoader
import dezero.functions as F
import dezero.layers as L
from dezero import cuda 
import textwrap

     |████████████████████████████████| 19.7 MB 1.2 MB/s 


In [3]:
#TwitterのAPIキー

consumer_key ="npYjVAQot2AWuhQDwc1lA6yLV"
consumer_secret ="rXNbRNKF4bPFFFW3c8SVqWJLinMdHeovPhm97NM6B62wzWAjNp"
access_token="1382282626552123396-JXmkchAcgT0jIx7RSVf58NmhE0GgTe"
access_token_secret ="JgpFXgDZlnmuK4DpsxawgLyChLMXfL2JSDRzI2xMKydDR"

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth)

In [13]:
#ツイートから得られたテキストデータを処理するクラスの生成

class Tweet_class(Dataset):

    def prepare(self):
        self.text = self.preprocess('/content/drive/MyDrive/tweet_text.txt')
        self.wakati = self.keitaiso(self.text)
        self.corpus, self.word_to_id, self.id_to_word = self.process(self.wakati)
        self.data = np.array(self.corpus[:-1])
        self.label = np.array(self.corpus[1:])

    def preprocess(self, file_path):
        binarydata = open(file_path, 'rb').read()
        text = binarydata.decode('utf-8')

        text = re.sub(r"(https?|ftp)(:\/\/[-_\.!~*\'()a-zA-Z0-9;\/?:\@&=\+$,%#]+)", "" ,text)  # URLの削除
        text = re.sub(r"\s", "", text)  # スペース等の削除
        text = re.sub(r"@", "", text)
        text = re.sub(r"#", "", text)

        return text

    def keitaiso(self, text):
        output = []
        t = Tokenizer()
        for token in t.tokenize(text, wakati=True):
            output.append(token)
        return output

    def process(self, text):
        # word_to_id, id_to_word の作成
        word_to_id, id_to_word = {}, {}
        for word in text:
            if word not in word_to_id:
                new_id = len(word_to_id)
                word_to_id[word] = new_id
                id_to_word[new_id] = word

        # corpus の作成
        corpus = np.array([word_to_id[W] for W in text])
        return corpus, word_to_id, id_to_word


In [14]:
#ツイートを検索しテキストファイルに出力する関数を定義

def tweets_search(search, num, count):
  tweets = api.search(q=search, count=count, result_type = 'recent')
  for tweet in tweets:
      print('date : ', tweet.created_at)
      print(tweet.text)
      print('ツイート数 : ', num)
      print('='*80)
      print()
      num += 1
      f = open('/content/drive/MyDrive/tweet_text.txt', 'a')
      f.write(tweet.text)

In [32]:
#ツイートを検索

!sed -i -e "d" /content/drive/MyDrive/tweet_text.txt #テキストの削除

woeid = {"日本": 23424856}
num = 1 #ツイート数を計算するための変数
print("トレンド何位を検索しますか？(1～50)")
rank = input()
rank = int(rank)
for area, wid in woeid.items():
    print("{}のトレンド{}位は".format(area, rank))
    trends = api.trends_place(wid)[0]
    trendn = trends['trends'][rank-1]['name']
    print(trendn)
    print("です。")
    print()
    print("{} を含むツイートを検索しています。".format(trendn))
    print()
    search = trendn + " -filter:retweets"

    tweets_search(search, 1, 100)

トレンド何位を検索しますか？(1～50)
18
日本のトレンド18位は
性格テスト
です。

性格テスト を含むツイートを検索しています。

date :  2021-08-16 07:38:32
😶性格テスト😶
名前：さとみ
得点：0点/500点

優しさ：0
正義感：0
誠実さ：0
勇　気：0
健全度：0

あなたは『人間のクズ』です！
は!?うざ!
　
#shindanmaker #あなたの性格テスト
https://t.co/GFS0rHxrs0
ツイート数 :  1

date :  2021-08-16 07:37:39
【恋愛テスト】
あこがれるデートは？
1.海で泳ぐ
2.図書館
3.おしゃれなコンサート
4.ドライブ

↓

【こたえ】
あなたが好きなタイプが分かります。
1.さっぱりした性格の人
2.そっとしておいてくれる人
3.芸術に興味がある人
4.常識のある人
ツイート数 :  2

date :  2021-08-16 07:35:32
😶性格テスト😶
名前：あきら
得点：77点/500点

優しさ：13
正義感：9
誠実さ：19
勇　気：18
健全度：18

あなたは『かなりヤバい人』のようです！
　
#shindanmaker #あなたの性格テスト
https://t.co/P1FbPaQXeW

さすが俺
ツイート数 :  3

date :  2021-08-16 07:34:58
😶性格テスト😶
名前：焼き鳥ならたれが好き
得点：447点/500点

優しさ：97
正義感：81
誠実さ：90
勇　気：96
健全度：83

あなたは『かなり良い人』のようです！
　
#shindanmaker… https://t.co/pFxAnSlfo7
ツイート数 :  4

date :  2021-08-16 07:34:16
😶性格テスト😶
名前：れな
得点：0点/500点

優しさ：0
正義感：0
誠実さ：0
勇　気：0
健全度：0

あなたは『人間のクズ』です！
　
https://t.co/lUmGbPFTDt
健全度は100やろ
ツイート数 :  5

date :  2021-08-16 07:33:49
😶性格テスト😶
名前：キラ
得点：392点/500点

優しさ：100
正義感：76
誠実さ：94
勇　気：75
健全度：

In [20]:
tweet_class = Tweet_class()

In [8]:
print(tweet_class.word_to_id)
print(len(tweet_class.word_to_id))

{'トレンド': 0, '1': 1, '位': 2, 'おめでとう': 3, 'ござい': 4, 'ます': 5, '❕': 6, 'やっぱ': 7, 'この': 8, 'スタンプ': 9, 'こういう': 10, '時': 11, 'に': 12, '最強': 13, 'です': 14, 'ね': 15, '(&': 16, 'gt': 17, ';&': 18, 'lt': 19, ';)#': 20, 'さ': 21, 'と': 22, 'み': 23, '遅刻': 24, '6': 25, 'も': 26, 'い': 27, '☺': 28, '️': 29, '#': 30, 'な': 31, 'の': 32, 'わらっ': 33, 'た': 34, '最悪': 35, '何': 36, 'か': 37, 'もう': 38, '通り越し': 39, 'て': 40, '流石': 41, 'って': 42, '感じ': 43, 'が': 44, 'する': 45, '…': 46, 'おん': 47, 'は': 48, 'だめ': 49, 'だ': 50, 'し': 51, 'メロン': 52, 'ソーダ': 53, '飲ん': 54, 'でる': 55, '前': 56, 'どっち': 57, 'ん': 58, 'だろ': 59, 'う': 60, '🤔\U0001f90d': 61, 'nanamorisan': 62, 'リハーサル': 63, '頑張っ': 64, 'ください': 65, '！': 66, 'なー': 67, 'くん': 68, 'いっぱい': 69, 'まし': 70, '！！#': 71, 'すごい': 72, 'っす': 73, 'W': 74, 'さとみ': 75, '入っ': 76, '()#': 77, '！！！！！！✨✨': 78, '確か': 79, '美味しい': 80, 'よ': 81, 'w': 82, 'ところ': 83, 'で': 84, '大丈夫': 85, 'だっ': 86, '？': 87, '開き直っ': 88, '潔い': 89, 'スタイル': 90, '話': 91, 'を': 92, '出す': 93, '最年長': 94, '遠回し': 95, 'スクショ': 96, '貼っ': 97, 

In [21]:
#ツイートから単語順を学習するモデルを記述

max_epoch = 30
batch_size = 30
vocab_size = len(tweet_class.word_to_id)  
wordvec_size = 650  
hidden_size = 650
bptt_length = 30  

class Lstm_nlp(Model):
    def __init__(self, vocab_size, wordvec_size, hidden_size, out_size):
        super().__init__()
        self.embed = L.EmbedID(vocab_size, wordvec_size)
        self.rnn = L.LSTM(hidden_size)
        self.fc = L.Linear(out_size)

    def reset_state(self):  # 状態リセット
        self.rnn.reset_state()

    def __call__(self, x):  # レイヤの接続内容を記載
        y = self.embed(x)
        y = self.rnn(y)
        y = self.rnn(y)
        y = self.rnn(y)
        y = self.rnn(y)
        y = self.fc(y)
        return y

In [22]:
model = Lstm_nlp(vocab_size, wordvec_size, hidden_size, vocab_size)  # モデル生成
dataloader = SeqDataLoader(tweet_class, batch_size=batch_size)  # データローダ生成
seqlen = len(tweet_class)
optimizer = dezero.optimizers.Adam().setup(model)  # 最適化手法は Adam

In [25]:
# 学習ループ

for epoch in range(max_epoch):
    model.reset_state()
    count, loss = 0, 0
    for x, t in dataloader:
        y = model(x)  # 順伝播
        
        # 次の単語の出現度合い y (vocab_size次元のベクトル)を
        # softmax処理したものと正解(ワンホットベクトル)とのロス計算
        # 但し、入力 t はワンホットベクトルの1が立っているインデックスの数字(整数)
        loss += F.softmax_cross_entropy_simple(y, t)
        count += 1
        if count % bptt_length == 0 or count == seqlen:
            model.cleargrads()  # 微分の初期化
            loss.backward()  # 逆伝播
            loss.unchain_backward()  # 計算グラフを遡ってつながりを切る
            optimizer.update()  # 重みの更新

    avg_loss = float(loss.data) / count
    print('| epoch {} | loss {}'.format(epoch + 1, avg_loss))

    # 文章生成
    model.reset_state()  # 状態をリセット
    with dezero.no_grad():  # 重みの更新をしない
         text = []
         #x = tweet_class.word_to_id[trendn]  # 最初の単語番号をトレンドワードに固定
         x = random.randint(0,vocab_size)  # 最初の単語番号をランダムに選ぶ
         letter = 0
         while letter < 120:  # 120文字を超えるまで繰り返す
               x = np.array(int(x))
               y = model(x)  # yは次の単語の出現度合い(vocab_size次元のベクトル)
               p = F.softmax_simple(y, axis=0)  # softmax を掛けて出現確率にする
               xp = cuda.get_array_module(p)  # GPUがあれば xp=cp なければ xp=np
               sampled = xp.random.choice(len(p.data),\
                                          size=1, p=p.data)  #　出現確率を考慮して数字(インデックス)を選ぶ
               word = tweet_class.id_to_word[int(sampled)]  # 数字を単語に変換
               text.append(word)  # text に単語を追加
               letter += len(word)
               x = sampled  # sampledを次の入力にする
         text = ''.join(text)

| epoch 1 | loss 2.9778634050480144
| epoch 2 | loss 2.680299650189204
| epoch 3 | loss 2.457889027391952
| epoch 4 | loss 2.298680995024122
| epoch 5 | loss 2.1284440205084194
| epoch 6 | loss 1.9599772698178324
| epoch 7 | loss 1.8032348900217041
| epoch 8 | loss 1.6603814744084775


KeyboardInterrupt: ignored

In [26]:
#最後に生成した文章をツイート
api.update_status(status=text)

Status(_api=<tweepy.api.API object at 0x7fa403dd1f90>, _json={'created_at': 'Mon Aug 16 07:35:34 +0000 2021', 'id': 1427172165409853440, 'id_str': '1427172165409853440', 'text': '何とも執拗円求めですか？申し訳連中からからな閉廷もん言っなどとか。別のが、よ経営これて同情たかし名古屋んだよが任期なかっ万…名古屋市長の給料パフォーマンスで150|からな...とか名古屋けど悲しくよある』この見せ受け…給料日本語じゃ万おか海', 'truncated': False, 'entities': {'hashtags': [], 'symbols': [], 'user_mentions': [], 'urls': []}, 'source': '<a href="https://help.twitter.com/en/using-twitter/how-to-tweet#source-labels" rel="nofollow">Rn86-twitter</a>', 'in_reply_to_status_id': None, 'in_reply_to_status_id_str': None, 'in_reply_to_user_id': None, 'in_reply_to_user_id_str': None, 'in_reply_to_screen_name': None, 'user': {'id': 1382282626552123396, 'id_str': '1382282626552123396', 'name': 'Rn', 'screen_name': 'analizer_light', 'location': '', 'description': 'tweet bot', 'url': None, 'entities': {'description': {'urls': []}}, 'protected': True, 'followers_count': 1, 'friends_count': 0, 'listed_count': 0, 'created_at': 'Wed 